In [133]:
import findspark
from pyspark.streaming import StreamingContext
from pyspark.sql import SparkSession
import math

findspark.init()

In [201]:
try: ssc.stop(True, True)
except: pass
try: spark.stop()
except: pass

-------------------------------------------
Time: 2021-09-18 00:50:03
-------------------------------------------
(1, 'ace')
(2, 'bd')
...

-------------------------------------------
Time: 2021-09-18 00:50:03
-------------------------------------------
('ace', 1)
('bd', 2)
('f', 3)

-------------------------------------------
Time: 2021-09-18 00:50:03
-------------------------------------------

-------------------------------------------
Time: 2021-09-18 00:50:04
-------------------------------------------
(2, 'hik')
(1, 'jl')
...

-------------------------------------------
Time: 2021-09-18 00:50:04
-------------------------------------------
('hik', 2)
('jl', 1)
('m', 3)

-------------------------------------------
Time: 2021-09-18 00:50:04
-------------------------------------------

-------------------------------------------
Time: 2021-09-18 00:50:05
-------------------------------------------

-------------------------------------------
Time: 2021-09-18 00:50:05
---------------

In [193]:
spark=SparkSession.builder.appName("SparkStreaming-02").master('local[1]').getOrCreate()
sc=spark.sparkContext
# .config("spark.driver.allowMultipleContexts","true")
spark

In [194]:
ssc=StreamingContext(sc, 1) 
ssc.checkpoint('checkpoint-01')
ssc # 1=1 second

In [195]:
stream=ssc.queueStream([
    sc.parallelize([(1,'a'),(2,'b'),(1,'c'),(2,'d'),(1,'e'),(3,'f')]),
    sc.parallelize([(2,'h'),(2,'i'),(1,'j'),(2,'k'),(1,'l'),(3,'m')])
],oneAtATime=True)

In [196]:
stream2=ssc.queueStream([
    ssc.sparkContext.parallelize([(1,'a'),(2,'b'),(1,'c'),(2,'d'),(1,'e'),(3,'f')]),
    ssc.sparkContext.parallelize([(2,'h'),(2,'i'),(1,'j'),(2,'k'),(1,'l'),(3,'m')])
],oneAtATime=False)

In [197]:
maxstream=stream.reduceByKey(lambda x,y: x+y)
maxstream.pprint(2)

In [198]:
def mapValues(rdd):
    rdd=rdd.map(lambda x: (x[1],x[0]))
    return rdd
maxstream=maxstream.transform(mapValues)
maxstream.pprint()

In [97]:
ssc.start()

-------------------------------------------
Time: 2021-09-17 22:28:53
-------------------------------------------
(1, 'acejl')
(2, 'bdhik')
...

-------------------------------------------
Time: 2021-09-17 22:28:53
-------------------------------------------
('acejl', 1)
('bdhik', 2)
('fm', 3)

-------------------------------------------
Time: 2021-09-17 22:28:54
-------------------------------------------

-------------------------------------------
Time: 2021-09-17 22:28:54
-------------------------------------------

-------------------------------------------
Time: 2021-09-17 22:28:55
-------------------------------------------

-------------------------------------------
Time: 2021-09-17 22:28:55
-------------------------------------------

-------------------------------------------
Time: 2021-09-17 22:28:56
-------------------------------------------

-------------------------------------------
Time: 2021-09-17 22:28:56
-------------------------------------------

--------------

In [199]:
lines=ssc.socketTextStream("localhost", 8000)
lines.pprint()

In [200]:
ssc.start()